In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import base64
from scipy.stats import linregress

data = pd.read_csv(r'./diabetes_prediction_dataset - diabetes_prediction_dataset.csv')

app = dash.Dash(__name__)
app.title = "Diabetes Prediction Dashboard"

app.layout = html.Div(
    children=[
        html.H1("Diabetes Prediction Dashboard", style={'text-align': 'center'}),
        html.Label("Select Chart:"),
        dcc.Dropdown(
            id='chart-dropdown',
            options=[
                {'label': 'Diabetes Histogram', 'value': 'hist_diabetes'},
                {'label': 'Average Blood Glucose by Diabetes', 'value': 'avg_glucose'},
                {'label': 'Average BMI by Diabetes', 'value': 'avg_bmi'},
                {'label': 'Diabetes Percentage by Smoking History', 'value': 'smoking_diabetes'},
                {'label': 'Heart Disease Percentage by Diabetes', 'value': 'heart_disease_diabetes'},
                {'label': 'Gender Distribution (Diabetes = 1)', 'value': 'gender_distribution'},
                {'label': 'Blood Glucose by Smoking History', 'value': 'glucose_smoking'},
                {'label': 'Bubble Plot: Age vs Blood Glucose Level', 'value': 'bubble_plot'},
                {'label': 'Scatter Plot with Regression Line', 'value': 'scatter_plot'},
                {'label': 'Bar Plot: Age by Heart Disease', 'value': 'bar_heart_disease'}
            ],
            value='hist_diabetes',
            style={'width': '50%', 'margin': 'auto'}
        ),
        html.Div(id='output-container', className='output-container')
    ]
)

def matplotlib_to_dash(fig):
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches='tight')
    buf.seek(0)
    encoded_image = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    return html.Img(src=f"data:image/png;base64,{encoded_image}", style={'width': '60%', 'height': 'auto', 'margin': 'auto'})

@app.callback(
    Output('output-container', 'children'),
    Input('chart-dropdown', 'value')
)
def update_chart(selected_chart):
    if selected_chart == 'hist_diabetes':
        fig, ax = plt.subplots()
        ax.hist(data['diabetes'], bins=10, color='skyblue')
        ax.set_title("Diagnosed with Diabetes")
        ax.set_xlabel("Diabetes")
        ax.set_ylabel("Frequency")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'avg_glucose':
        avg_blood_glucose = data.groupby('diabetes')['blood_glucose_level'].mean()
        fig, ax = plt.subplots()
        ax.plot(avg_blood_glucose.index, avg_blood_glucose.values, marker='o')
        ax.set_title("Average Blood Glucose by Diabetes")
        ax.set_xlabel("Diabetes")
        ax.set_ylabel("Average Blood Glucose Level")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'avg_bmi':
        bmi_mean = data.groupby('diabetes')['bmi'].mean()
        fig, ax = plt.subplots()
        ax.plot(bmi_mean.index, bmi_mean.values, marker='o')
        ax.set_title("Average BMI by Diabetes")
        ax.set_xlabel("Diabetes")
        ax.set_ylabel("Average BMI")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'smoking_diabetes':
        smoking_diabetes_percentage = data.groupby('smoking_history')['diabetes'].mean() * 100
        fig, ax = plt.subplots()
        ax.plot(smoking_diabetes_percentage.index, smoking_diabetes_percentage.values, marker='o')
        ax.set_title("Diabetes Percentage by Smoking History")
        ax.set_xlabel("Smoking History")
        ax.set_ylabel("Diabetes Percentage (%)")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'heart_disease_diabetes':
        heart_disease_percentage = data.groupby('diabetes')['heart_disease'].mean() * 100
        fig, ax = plt.subplots()
        ax.plot(heart_disease_percentage.index, heart_disease_percentage.values, marker='o')
        ax.set_title("Heart Disease Percentage by Diabetes")
        ax.set_xlabel("Diabetes")
        ax.set_ylabel("Heart Disease Percentage (%)")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'gender_distribution':
        diabetes_data = data[data['diabetes'] == 1]
        gender_counts = diabetes_data['gender'].value_counts()
        fig, ax = plt.subplots()
        ax.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90, colors=['skyblue', 'lightcoral'])
        ax.set_title("Gender Distribution (Diabetes = 1)")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'glucose_smoking':
        fig, ax = plt.subplots()
        sns.barplot(x='smoking_history', y='blood_glucose_level', data=data, ci='sd', palette='viridis', ax=ax)
        ax.set_title("Blood Glucose by Smoking History")
        ax.set_xlabel("Smoking History")
        ax.set_ylabel("Blood Glucose Level")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'bubble_plot':
        fig, ax = plt.subplots()
        sns.scatterplot(x='age', y='blood_glucose_level', size=data['bmi'], hue='smoking_history', data=data, sizes=(20, 200), ax=ax)
        ax.set_title("Bubble Plot: Age vs Blood Glucose Level")
        ax.set_xlabel("Age")
        ax.set_ylabel("Blood Glucose Level")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'scatter_plot':
        slope, intercept, r_value, _, _ = linregress(data['age'], data['blood_glucose_level'])
        fig, ax = plt.subplots()
        ax.scatter(data['age'], data['blood_glucose_level'], color='blue', alpha=0.6)
        ax.plot(data['age'], slope * data['age'] + intercept, color='red')
        ax.set_title("Scatter Plot with Regression Line")
        ax.set_xlabel("Age")
        ax.set_ylabel("Blood Glucose Level")
        return matplotlib_to_dash(fig)

    elif selected_chart == 'bar_heart_disease':
        fig, ax = plt.subplots()
        ax.bar(data['age'], data['heart_disease'], color='orange')
        ax.set_title("Age by Heart Disease Level")
        ax.set_xlabel("Age")
        ax.set_ylabel("Heart Disease Level")
        return matplotlib_to_dash(fig)

if __name__ == "__main__":
    app.run_server(debug=True)
